In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
import pytz



In [4]:
# Set the timezone to IST
IST = pytz.timezone('Asia/Kolkata')


In [5]:

# Get the current time in IST
now_ist = datetime.now(IST)


In [9]:

# Check if the current hour is between 1 PM (13) and 2 PM (14)
if 13 <= now_ist.hour < 14:
    # Load the dataset
    try:
        df = pd.read_csv('Play Store Data.csv')

        # --- Data Cleaning and Preprocessing ---

        # 1. Handle 'Installs'
        df['Installs'] = df['Installs'].str.replace('[,+]', '', regex=True)
        df = df[df['Installs'] != 'Free']
        df['Installs'] = pd.to_numeric(df['Installs'])


        # 2. Handle 'Price' and calculate 'Revenue'
        df['Price'] = df['Price'].str.replace('$', '', regex=False)
        df['Price'] = pd.to_numeric(df['Price'], errors='coerce')
        df['Revenue'] = df['Installs'] * df['Price']

        # 3. Handle 'Size'
        def convert_size(size):
            if isinstance(size, str):
                if 'M' in size:
                    return float(size.replace('M', ''))
                elif 'k' in size:
                    return float(size.replace('k', '')) / 1024
            return np.nan

        df['Size'] = df['Size'].apply(convert_size)

        # 4. Handle 'Android Ver'
        df['Android Ver'] = df['Android Ver'].str.extract(r'(\d+\.\d+)').astype(float)


        # --- Filtering the Data ---
        df_filtered = df[
            (df['Installs'] >= 10000) &
            (df['Revenue'] >= 10000) &
            (df['Android Ver'] > 4.0) &
            (df['Size'] > 15) &
            (df['Content Rating'] == 'Everyone') &
            (df['App'].str.len() <= 30)
        ]

        # --- Top 3 Categories ---
        top_categories = df_filtered['Category'].value_counts().nlargest(3).index.tolist()
        df_top = df_filtered[df_filtered['Category'].isin(top_categories)]

        # --- Data Aggregation for Plotting ---
        chart_data = df_top.groupby(['Category', 'Type']).agg(
            Average_Installs=('Installs', 'mean'),
            Average_Revenue=('Revenue', 'mean')
        ).reset_index()

        # Pivot the data for plotting
        pivot_data = chart_data.pivot(index='Category', columns='Type', values=['Average_Installs', 'Average_Revenue'])

        # --- Chart Creation ---
        fig, ax1 = plt.subplots(figsize=(12, 7))

        # Bar chart for Average Installs
        pivot_data['Average_Installs'].plot(kind='bar', ax=ax1, position=0, width=0.4, color=['#1f77b4', '#ff7f0e'])
        ax1.set_ylabel('Average Installs', color='black')
        ax1.tick_params(axis='y', labelcolor='black')
        ax1.set_xlabel('App Category')
        ax1.set_xticklabels(ax1.get_xticklabels(), rotation=45, ha="right")


        # Line chart for Average Revenue on a secondary y-axis
        ax2 = ax1.twinx()
        pivot_data['Average_Revenue'].plot(kind='line', ax=ax2, marker='o', color=['#2ca02c', '#d62728'])
        ax2.set_ylabel('Average Revenue ($)', color='black')
        ax2.tick_params(axis='y', labelcolor='black')

        # --- Final Touches ---
        plt.title('Average Installs vs. Revenue for Top 3 Categories (Free vs. Paid Apps)')
        fig.tight_layout()
        plt.legend(title='Type')
        plt.grid(True)
        plt.savefig('dual_axis_chart.png')

        print("Chart generated successfully and saved as dual_axis_chart.png")

    except FileNotFoundError:
        print("Error: 'Play Store Data.csv' not found. Please make sure the file is in the correct directory.")
    except Exception as e:
        print(f"An error occurred: {e}")

else:
    print("Chart generation is only available between 1 PM and 2 PM IST.")
    print(f"Current IST time: {now_ist.strftime('%Y-%m-%d %H:%M:%S')}")

Chart generation is only available between 1 PM and 2 PM IST.
Current IST time: 2025-08-20 19:35:48
